#### The model decides pre-disaster procurement quantities under long-term quantity-flexibility contracts, post-disaster relief orders via dynamic option contracts, and adjustable warehousing capacity, all while hedging against supply disruptions and demand uncertainty. The first stage (pre-disaster) determines contract quantities and warehouse capacity; the second stage (post-disaster, across scenarios) allocates extra orders, uses backup suppliers, satisfies scenario demands, and incurs penalties for unmet demand or surplus disposal.


### Decision variables

$I$ index main suppliers,
$J$ backup suppliers,
$T$ time periods,
$S$ disaster scenarios with $p_{s}$ probability

First Stage
$x_{i,t}$ >=0 : quantity contracted with main supplier $i$ for period $t$.

$w_{t}$ >=0 : contracted warehouse capacity for period $t$.

Second Satge

$y^s_{j,t}$ >=0 : extra quantity taken from main supplier $i$ in scenario $s$.

$z^s_{j,t}$ >=0 : quantity ordered from backup supplier $j$ in scenario $s$.

$u^s_{t}$ >=0 : unmet demand (shortage) in period $t$ under scenario $s$.

$δ^s_{t}$ >=0 : unused inventory disposed in period $t$.


### Objective Function

$$
\min \sum_{t \in T} \left( \sum_{i \in I} c_{i,t}^x x_{i,t} + h_t w_t \right) + \sum_{s \in S} p_s \left[ \sum_{t \in T} \left( \sum_{i \in I} c_{i,t}^y y_{i,t}^s + \sum_{j \in J} c_{j,t}^z z_{j,t}^s + \pi_t u_t^s + \rho_t \delta_t^s \right) \right]
$$

$$
\begin{aligned}
&- c_{i,t}^{x}, c_{i,t}^{y} \text{ are main-supplier pre- and post-disaster unit costs}; \\
&- c_{j,t}^{z} \text{ are backup-supplier option premiums}; \\
&- h_t \text{ is per-unit warehouse capacity cost}; \\
&- \pi_t, \rho_t \text{ are penalties for unmet demand and surplus disposal in period } t.\\
&- \rho_s \text{ probability of scenario $s$}.
\end{aligned}
$$


#### Constraints

#### 1. Pre-disaster capacity and flexibility

$$
x_{i,t} \leq \overline{X}_{i,t} \quad \text{and} \quad y_{i,t}^s \leq \gamma_{i,t} x_{i,t} \quad \forall i, t, s
$$

where $\overline{X}_{i,t}$ is supplier capacity and $\gamma_{i,t}$ is the flexibility ratio.

#### 2. Backup supplier limits

$$
z_{j,t}^s \leq \overline{Z}_{j,t} \quad \forall j, t, s
$$

#### 3. Warehouse capacity

$$
\sum_i x_{i,t} + \sum_s p_s (y_{i,t}^s + z_{j,t}^s) \leq w_t \quad \forall t
$$

#### 4. Demand satisfaction (per scenario)

$$
\sum_i (x_{i,t} + y_{i,t}^s) + \sum_j z_{j,t}^s + \delta_{t-1}^s - \delta_t^s + u_{t-1}^s - u_t^s \geq D_t^s \quad \forall t, s
$$

Ensuring flows balance demand $D_t^s$.

#### 5. Non-anticipativity

$x_{i,t}, w_t$ are identical across all $s$.


#### Importing libraries


In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Assuming Required data
I = ['A','B'] #main suppliers
J = ['C'] #Backup suppliers
T = [1,2] # Time periods
S = ['low','med','high']  # Scenarios

# Probabilities
p = {'low':0.5, 'med':0.3, 'high':0.2}

# Demand parameters
D = {('low',1):100, ('low',2):120,
     ('med',1):200, ('med',2):180,
     ('high',1):230,('high',2):240}

# Cost parameters
c_x = {('A',1):10, ('A',2):11, ('B',1):12, ('B',2):13}
c_y = {('A',1):12, ('A',2):13, ('B',1):14, ('B',2):15}
c_z = {('C',1):20, ('C',2):22}
h = {1:2, 2:2}
pi = {1:50, 2:50}
rho = {1:5, 2:5}

# Capacity parameters
X_cap = {('A',1):200, ('A',2):200, ('B',1):150, ('B',2):150}
Z_cap = {('C',1):100, ('C',2):100}
gamma = {('A',1):0.5, ('A',2):0.5, ('B',1):0.4, ('B',2):0.4}


In [3]:
m = gp.Model("relief")

Restricted license - for non-production use only - expires 2026-11-23


##### First Stage


In [4]:
x = m.addVars(I,T,lb=0, name='x') #Pre disaster procurement
w = m.addVars(T, lb=0, name='w') #warehouse capacity

##### Second Stage


In [5]:
y = m.addVars(S, I, T, lb=0, name="y")    # Post-disaster procurement
z = m.addVars(S, J, T, lb=0, name="z")    # Backup procurement
delta = m.addVars(S, T, lb=0, name="delta")  # Surplus
u = m.addVars(S, T, lb=0, name="u")        # Shortage

#### Objective function


In [6]:
first_stage = (
    gp.quicksum(c_x[i,t] * x[i,t] for i in I for t in T) +
    gp.quicksum(h[t] * w[t] for t in T)
)

second_stage = gp.quicksum(
    p[s] * (
        gp.quicksum(c_y[i,t] * y[s,i,t] for i in I for t in T) +
        gp.quicksum(c_z[j,t] * z[s,j,t] for j in J for t in T) +
        gp.quicksum(pi[t] * u[s,t] + rho[t] * delta[s,t] for t in T)
    ) for s in S
)

m.setObjective(first_stage + second_stage, gp.GRB.MINIMIZE)

#### Constraints


In [7]:
# 1. Pre-disaster capacity constraints
for i in I:
    for t in T:
        m.addConstr(x[i,t] <= X_cap[i,t], name=f"pre_cap_{i}_{t}")

In [8]:
# 2. Post-disaster flexibility constraints
for s in S:
    for i in I:
        for t in T:
            m.addConstr(y[s,i,t] <= gamma[i,t] * x[i,t], 
                        name=f"flex_{s}_{i}_{t}")

In [9]:
# 3. Backup supplier capacity constraints
for s in S:
    for j in J:
        for t in T:
            m.addConstr(z[s,j,t] <= Z_cap[j,t], 
                        name=f"backup_cap_{s}_{j}_{t}")


In [10]:
# 4. Warehouse capacity constraints (fixed)
for t in T:
    m.addConstr(
        gp.quicksum(x[i,t] for i in I) + 
        gp.quicksum(
            p[s] * (
                gp.quicksum(y[s,i,t] for i in I) + 
                gp.quicksum(z[s,j,t] for j in J)
            )
            for s in S
        ) <= w[t],
        name=f"warehouse_{t}"
    )


In [11]:
# 5. Demand satisfaction constraints (fixed)
for s in S:
    for t in T:
        lhs = (
            gp.quicksum(x[i,t] + y[s,i,t] for i in I) 
            + gp.quicksum(z[s,j,t] for j in J)
        )
        
        # Add inventory carry-over terms only if t > 1
        if t > 1:
            lhs += (delta[s,t-1] - delta[s,t] + u[s,t-1] - u[s,t])
        
        m.addConstr(lhs >= D[s,t], name=f"demand_{s}_{t}")

In [12]:
m.write("stochastic programming aghajani.lp")

In [13]:
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 30 rows, 36 columns and 100 nonzeros
Model fingerprint: 0xeaf3748c
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve removed 21 rows and 25 columns
Presolve time: 0.01s
Presolved: 9 rows, 11 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.1000000e+02   6.625000e+01   0.000000e+00      0s
       8    3.0086667e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.008666667e+03


In [14]:
if m.status == gp.GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Total Cost: ${m.ObjVal:.2f}\n")

    print("=== First-Stage Variables ===")
    # 1. Pre-disaster procurement (x)
    for i in I:
        for t in T:
            print(f"x[{i},{t}] = {x[i,t].X:.2f}")
    
    # 2. Warehouse capacity (w)
    print("\nWarehouse Capacity:")
    for t in T:
        print(f"w[{t}] = {w[t].X:.2f}")
    
    print("\n=== Second-Stage Variables ===")
    # 3. Post-disaster procurement (y)
    print("\nPost-disaster Procurement (y):")
    for s in S:
        for i in I:
            for t in T:
                print(f"y[{s},{i},{t}] = {y[s,i,t].X:.2f}")
    
    # 4. Backup procurement (z)
    print("\nBackup Procurement (z):")
    for s in S:
        for j in J:
            for t in T:
                print(f"z[{s},{j},{t}] = {z[s,j,t].X:.2f}")
    
    # 5. Surplus (delta) and shortage (u)
    print("\nSurplus (delta):")
    for s in S:
        for t in T:
            print(f"delta[{s},{t}] = {delta[s,t].X:.2f}")
    
    print("\nShortage (u):")
    for s in S:
        for t in T:
            print(f"u[{s},{t}] = {u[s,t].X:.2f}")
else:
    print("No optimal solution found")

Optimal solution found!
Total Cost: $3008.67

=== First-Stage Variables ===
x[A,1] = 133.33
x[A,2] = 0.00
x[B,1] = 0.00
x[B,2] = 0.00

Warehouse Capacity:
w[1] = 172.67
w[2] = 0.00

=== Second-Stage Variables ===

Post-disaster Procurement (y):
y[low,A,1] = 0.00
y[low,A,2] = 0.00
y[low,B,1] = 0.00
y[low,B,2] = 0.00
y[med,A,1] = 66.67
y[med,A,2] = 0.00
y[med,B,1] = 0.00
y[med,B,2] = 0.00
y[high,A,1] = 66.67
y[high,A,2] = 0.00
y[high,B,1] = 0.00
y[high,B,2] = 0.00

Backup Procurement (z):
z[low,C,1] = 0.00
z[low,C,2] = 0.00
z[med,C,1] = 0.00
z[med,C,2] = 0.00
z[high,C,1] = 30.00
z[high,C,2] = 0.00

Surplus (delta):
delta[low,1] = 120.00
delta[low,2] = 0.00
delta[med,1] = 180.00
delta[med,2] = 0.00
delta[high,1] = 240.00
delta[high,2] = 0.00

Shortage (u):
u[low,1] = 0.00
u[low,2] = 0.00
u[med,1] = 0.00
u[med,2] = 0.00
u[high,1] = 0.00
u[high,2] = 0.00
